In [18]:
import sqlite3 
import pandas as pd 
import requests
import json  
from datetime import datetime
import secrets
import telegram #pip install python-telegram-bot --upgrade
import time
import math
import numpy as np

pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_columns', 500) 
pd.set_option('display.max_rows', 500) 

def connect_create_table(name_db='db.db'):
    conn = sqlite3.connect(name_db)  # You can create a new database by changing the name within the quotes
    c = conn.cursor() # The database will be saved in the location where your 'py' file is saved

    # Create table - CLIENTS
    c.execute('''CREATE TABLE if not exists POSITIONS
                 ([id] INTEGER PRIMARY KEY,
                 [date] DATETIME,
                 [encryptedUid] TEXT,
                 [nick] TEXT,
                 [symbol] TEXT,
                 [status] TEXT,
                 [entryPrice] FLOAT,
                 [markPrice] FLOAT,
                 [pnl] FLOAT,
                 [roe] FLOAT,
                 [amount] FLOAT,
                 [amount_usd] FLOAT,
                 [amount_percent] FLOAT,
                 [updateTimeStamp] TEXT,
                 [trader_max_size] FLOAT,
                 [request_id] INT
                 )''')
    conn.commit()

connect_create_table()

In [19]:
def human_format(num):
    num = float(num)
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    # add more suffixes if you need them
    return '%.2f%s' % (num, ['', 'K', 'M', 'G', 'T', 'P'][magnitude])

In [20]:
import mysql.connector

def insert_mysql(trader_info, request_id, vector, status):
  mydb = mysql.connector.connect(
    host="localhost",
    user="montep",
    password="autograder",
    database="copytrading"
  )
  encryptedUid = trader_info[0]
  nick         = trader_info[1]
  try:
    symbol      = vector[0]
    entryPrice  = float(vector[1])
    markPrice   = float(vector[2])
    pnl         = float(vector[3])
    row         = float(vector[4])
    try:
      amount = float(vector[5])
    except:
      amount = 0
    trader_max_size = vector[6]
    if trader_max_size == 0:
      trader_max_size = 1
    amount_usd  = round(markPrice * amount,0)
    amount_percent = amount_usd / trader_max_size
    if amount_percent == "NULL":
      amount_percent = 0
    updateTimestamp = time.time()
    date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    #############################################
    a1=(date)
    a2=(encryptedUid)
    a3=(nick)
    a4=(symbol)
    a5=(status)
    a6=(entryPrice)
    a7=(markPrice)
    a8=(pnl)
    a9=(row)
    a10=(amount)
    a11=(amount_usd)
    a12=float(amount_percent)
    a13=(updateTimestamp)
    a14=float(trader_max_size)
    a15=int(request_id)
    ##############################################
    k=(a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13,a14,a15)
    # print (date, encryptedUid, nick, symbol, status, entryPrice, markPrice, pnl, row, amount, amount_usd, amount_percent, updateTimestamp, trader_max_size, request_id)
    mycursor = mydb.cursor()
    sql = "INSERT INTO positions (date, encryptedUid, nick, symbol, status, entryPrice, markPrice, pnl, row, amount, amount_usd, amount_percent, updateTimeStamp, trader_max_size, request_id) VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    val = (k[0],k[1],k[2],k[3],k[4],k[5],k[6],k[7],k[8],k[9],k[10],k[11],k[12],k[13],k[14])
    mycursor.execute(sql, val)
    mydb.commit()
    # print(mycursor.rowcount, "record inserted.")
  except mysql.connector.Error as err:
    print("Something went wrong...: {}".format(err))

In [21]:
def insert_in_table(trader_info, request_id, vector, status):
    encryptedUid = trader_info[0]
    nick         = trader_info[1]
    try:
        sqliteConnection = sqlite3.connect('db.db')
        cursor = sqliteConnection.cursor()
        
        symbol      = vector[0]
        entryPrice  = vector[1]
        markPrice   = vector[2]
        pnl         = vector[3]
        row         = vector[4]
        try: amount = float(vector[5])
        except: amount = 0
        trader_max_size = vector[6]
        if trader_max_size == 0: trader_max_size = 1
        
        amount_usd  = round(markPrice * amount,0)
        amount_percent = amount_usd / trader_max_size
        if amount_percent == "NULL":
            amount_percent = 0
        updateTimestamp = time.time()
        date        = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        # Need to add more info of student (such as the txns)
        sqlite_insert_with_param = """INSERT INTO POSITIONS 
                          (date, encryptedUid, nick, symbol, status, entryPrice, markPrice, pnl, roe, amount, amount_usd,amount_percent, updateTimeStamp,trader_max_size, request_id) 
                          VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?,?,?,?,?,?);"""

        data_tuple = (date, encryptedUid, nick, symbol, status, entryPrice, markPrice, pnl, row, amount, amount_usd,amount_percent, updateTimestamp,trader_max_size,request_id) 
        cursor.execute(sqlite_insert_with_param, data_tuple)
        sqliteConnection.commit()

        cursor.close()

    except sqlite3.Error as error:
        print("Failed to insert Python variable into sqlite table", error)
    finally:
        if sqliteConnection:
            sqliteConnection.close()

In [22]:
def do_query(db="db.db", query="SELECT * FROM POSITIONS"):    
    con = sqlite3.connect(db)
    df = pd.read_sql_query(query, con)
    
    return df

In [23]:
def get_positions(trader):
    endpoint = "https://www.binance.com/bapi/futures/v1/public/future/leaderboard/getOtherPosition"
    params   =  {"encryptedUid":trader,  "tradeType":"PERPETUAL"}
    headers  =  {"content-type":"application/json;charset=UTF-8"}
    response = requests.post(endpoint, json=params, headers=headers) #Add proxies = proxyDict # Attention to JSON = PARAMS
    result   = response.json()
    
    return result

In [24]:
def send_message(text, chat_id = "-1001455341077"):
    my_bot  = telegram.Bot("814569733:AAGmMk_P4p8EdLkj-fj2fVziyByxHIcJpm0")
    my_bot.send_message(chat_id=chat_id, text=text, parse_mode = "Markdown")
    

In [25]:
def send_message_discord(text):
    #Webhook of my channel. Click on edit channel --> Webhooks --> Creates webhook
    webhook = "https://discord.com/api/webhooks/892399337731133500/_IE9zchjLp3lRrImBt8sp_YgrI8A9Kan1Vc6wv7eY7vvkVNEAGF-YkVkiUF4pKxT5J3R"
    data    = {"content": text}
    response = requests.post(webhook, json=data)

In [26]:
def get_current_price(symbol):
    info = requests.get(f"https://fapi.binance.com/fapi/v1/ticker/24hr?symbol={symbol}").json()
    price = float(info["lastPrice"])
    return price

In [27]:
def create_message(trader, row):
    mydb = mysql.connector.connect(
        host="localhost",
        user="montep",
        password="autograder",
        database="copytrading"
        )

    if row["difference"] > 0: trade_type = "BUY"
    else: trade_type = "SELL" 
        
    status_new       = str(row["status_new"])
    status_old       = str(row["status_old"])
    if status_new =="OPEN" and status_old =="OPEN": action = "Modify existent trade"
    if status_new =="OPEN" and status_old !="OPEN": action = "Open a new trade"
    if status_new !="OPEN" and status_old =="OPEN": action = "Close existent trade"
        
    trader_max_size  = row["trader_max_size_new"]
    new_position     = round(row["amount_new"],4)
    old_position     = round(row["amount_old"],4)
    price            = str(get_current_price(row["symbol"]))    # Avoid 0s for missing prices
    difference       = str(abs(row["difference"]))
    symbol           = str(row["symbol"])
    date             = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
   
    if (row["amount_usd_new"]== "NULL"):
        row["amount_usd_new"]=0
    if (row["amount_usd_old"]== "NULL"):
        row["amount_usd_old"]=0
    
    amount_usd_new       = row["amount_usd_new"]
    amount_usd_old       = row["amount_usd_old"]
    if(trader_max_size==0):
        new_position_percent = str(round(0.000))
        old_position_percent = str(round(0.000))
    else:
        new_position_percent = str(round(100* amount_usd_new / trader_max_size,3))
        old_position_percent = str(round(100* amount_usd_old / trader_max_size,3))
    delta_percent        = str(round(abs(float(new_position_percent) - float(old_position_percent)),3))
    
    print('.....................message..............................')
    message =  "💰" + trader + " " + date + "\n*" + action + "*\nPosition change from " + str(old_position) + symbol + " to " + str(new_position) + symbol + ".\n" + trade_type + " " + difference + symbol + " at " + price + "$" + "\nTrade went from " + old_position_percent + "% to "+ new_position_percent +"% (" + trade_type + " " + delta_percent + "%)"
    # msg =  str(trader) +  str(date) + str(action) + str(old_position) + symbol + str(new_position) + symbol + trade_type  + difference + symbol + price + old_position_percent + new_position_percent + trade_type +  delta_percent
    m = str(trader) + " " +str(date)+ "\n*" +str(action)+ "*\nPosition change from " + str(old_position) +str(symbol)+" to " + str(new_position) +str(symbol)+".\n " +str(trade_type)+ " " + str(difference)+ str(symbol)+ " at " +str(price)+ "$" + "\nTrade went from " + str(old_position_percent)+ "to "+ str(new_position_percent) +"(" + str(trade_type) + " " + str(delta_percent) + ")"
   
    a1=(str(trader))
    a2=(str(date))
    a3=(str(m))
   
            
    try:
        mycursor = mydb.cursor()
       
        sql = "INSERT INTO msg (traders, date, message) VALUES(%s,%s,%s)"
        val = (a1,a2,a3)
        mycursor.execute(sql, val)
        mydb.commit()
   

    except mysql.connector.Error as err:
        print("Something went wrong...: {}".format(err))


    return message

In [28]:
def message_logic(row, trader):
    elite_traders = ["TraderT","ADnefina","Nothingness","BBOS","ChinaPazzo","TradingHorse"]
    
    if row["difference"] != 0:   
        if   trader in elite_traders:  chat_id = -1001532365379
        elif trader == "ClickHereNow": chat_id = -591308509
        else:   chat_id = -530891051    
        message = create_message(trader, row)
        # print(message)

        try: 
            # send_message(message, chat_id)
            print(message)
        except Exception as e: print(e)
        # if trader in elite_traders : send_message_discord(message)
        time.sleep(2)

In [29]:
def get_positions_db(two_ids):
    # Get last update from trader.             
    this_id, last_id = two_ids["request_id"][0], two_ids["request_id"][1]

    # Get all the positions with that request_id
    new_positions   = do_query(query=f"SELECT * FROM POSITIONS WHERE request_id = '{this_id}'")
    old_positions   = do_query(query=f"SELECT * FROM POSITIONS WHERE request_id = '{last_id}'")

    # Compare the previous position with the new one, pair by pair
    both               = old_positions.merge(new_positions, "outer", on ="symbol", suffixes=('_old', '_new')).fillna(0) #Filling nas (if previous or new position is na, is because is not present, hence 0)
    both["difference"] = round(both["amount_new"] - both["amount_old"],4)
    # print(both[["symbol","amount_percent_old","amount_percent_new","difference", "amount_usd_new"]])
    
    return both

In [30]:
def get_trader_max_size(trader,tol = 0.20):
    start = time.time()
    trades        = do_query(query=f"select amount_usd, request_id from positions where encryptedUid = '{trader}'")
    trades["amount_usd"]    = abs(trades["amount_usd"])
    sum_positions = trades.groupby(['request_id'])["amount_usd"].sum()
    position_max  = sum_positions.max()
    trader_max_size = position_max*(1+tol)
    # print("time for getting trader max size:", round( time.time() - start,5))
    return trader_max_size

In [31]:
def get_sentiment():
    requests     = do_query(query=f"select distinct(request_id) from POSITIONS order by request_id desc limit {len(names)}")
    request_list = list(requests["request_id"])
    
    last_raw_positions = do_query(query=f"SELECT * FROM POSITIONS WHERE request_id in {tuple(request_list)}")
    
    sentiment = 100*pd.DataFrame(last_raw_positions.groupby(['symbol'])['amount_percent'].agg('mean'))
    sentiment = sentiment.iloc[1: , :]
    sentiment = sentiment.sort_values('amount_percent').dropna()
    
    return sentiment

In [32]:
def get_sentiment_message():
    sentiment = get_sentiment()
    sentiment = json.loads(sentiment.to_json())["amount_percent"]

    keys = list(sentiment.keys())
    values = list(sentiment.values())

    msg = ""
    for i in range(len(sentiment)):
        msg = msg + str(keys[i]) + ": " + str(round(values[i],2)) + "%\n"
        
    return msg

In [33]:
import mysql.connector

def traders_in_msql():
  mydb = mysql.connector.connect(
  host="localhost",
  user="montep",
  password="autograder",
  database="copytrading"
  )
  names    = {"CCF3E0CB0AAD54D9D6B4CEC5E3E741D2":"TraderT", 
              "49A7275656A7ABF56830126ACC619FEB":"ADnefina", 
              "A532C4316C00206168F795EDFBB3E164":"ClickHereNow", 
              "D3AFE978B3F0CD58489BC27B35906769":"CheerWindCard",
              "FB23E1A8B7E2944FAAEC6219BBDF8243":"BananaMan1", 
              "F318AE1B1BB3AEF4EDBD36E5AE250CD1":"BigPiePlayer", 
              "2F9D01E80429F97670415A5A5DDD9405":"CryptoPinoy",
              "95ACEF9493428B28CCADDA38A2127859":"ChurchChalkUnder",
              "382066A3597798ED9F69FECBCABAA00D":"911korn",
              "15A34DAA83C4B2D7F3A4BBAB4D2FECCA":"BakeAlsoShare",
              "3E478F9133714216B608E95B4ACF1FA3":"SABSAB",
              "B936AEC059917C5A730D45922A9F4334":"GeboyMujaer",
              "8D27A8FA0C0A726CF01A7D11E0095577":"Nothingness",
              "C6EC5B8B2D08F5748A1D8D3D849023A3":"GetSwifty",
              "191792DA7B3EEB10DAF1D51B6926F2E1":"Mikkiko",
            "7BFD2EBA42119D988CAA64F0BA6C64C2":"BBOS",
            "D64DDD2177FA081E3F361F70C703A562":"TradingHorse",
            "3480B1934B1E2F43E8B05301E73C6185":"TheBlueSky",
            "F92558003A58BECE1416F9D4FDE7CC77":"ThreeSurpriseTwelfth"
            }

  # names = {"A532C4316C00206168F795EDFBB3E164":"ClickHereNow"}

  traders=list(names.keys())
  did= list(names.keys())
  tradvalue  = list(names.values())
  num=len(did)
  try:
    for i in range(num):
      a=str(did[i])
      b=str(tradvalue[i])
      mycursor = mydb.cursor()
      sql = "INSERT INTO traders(keyid, names) VALUES (%s, %s)"
      val = (a,b)
      mycursor.execute(sql, val)
      mydb.commit()
  except:
    pass
    # print('data is not insert')  # mycursor.executemany(sql)

In [34]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="montep",
  password="autograder",
  database="copytrading"
)
names    = {"CCF3E0CB0AAD54D9D6B4CEC5E3E741D2":"TraderT", 
            "49A7275656A7ABF56830126ACC619FEB":"ADnefina", 
            "A532C4316C00206168F795EDFBB3E164":"ClickHereNow", 
            "D3AFE978B3F0CD58489BC27B35906769":"CheerWindCard",
            "FB23E1A8B7E2944FAAEC6219BBDF8243":"BananaMan1", 
            "F318AE1B1BB3AEF4EDBD36E5AE250CD1":"BigPiePlayer", 
            "2F9D01E80429F97670415A5A5DDD9405":"CryptoPinoy",
            "95ACEF9493428B28CCADDA38A2127859":"ChurchChalkUnder",
            "382066A3597798ED9F69FECBCABAA00D":"911korn",
            "15A34DAA83C4B2D7F3A4BBAB4D2FECCA":"BakeAlsoShare",
            "3E478F9133714216B608E95B4ACF1FA3":"SABSAB",
            "B936AEC059917C5A730D45922A9F4334":"GeboyMujaer",
            "8D27A8FA0C0A726CF01A7D11E0095577":"Nothingness",
            "C6EC5B8B2D08F5748A1D8D3D849023A3":"GetSwifty",
            "191792DA7B3EEB10DAF1D51B6926F2E1":"Mikkiko",
           "7BFD2EBA42119D988CAA64F0BA6C64C2":"BBOS",
           "D64DDD2177FA081E3F361F70C703A562":"TradingHorse",
           "3480B1934B1E2F43E8B05301E73C6185":"TheBlueSky",
           "F92558003A58BECE1416F9D4FDE7CC77":"ThreeSurpriseTwelfth"
           }

# names = {"A532C4316C00206168F795EDFBB3E164":"ClickHereNow"}

traders=list(names.keys())
# did= list(names.keys())
# tradvalue  = list(names.values())
# num=len(did)
# try:
#   for i in range(num):
#     a=str(did[i])
#     b=str(tradvalue[i])
#     mycursor = mydb.cursor()
#     sql = "INSERT INTO traders(keyid, names) VALUES (%s, %s)"
#     val = (a,b)
#     mycursor.execute(sql, val)
#     mydb.commit()
# except Exception as err:
#     print(err)
    # print("Something went wrong: {}".format(err))
name=list(names)
keys     = ["symbol", "entryPrice", "markPrice", "pnl", "roe", "amount"]

while True:
    start_trader = time.time()
    for trader in traders:
        # print(" ")
        # print("______________________", names[trader], "______________________")
        trader_info = [trader, names[trader]]    # Write the name of the trader
        trader_max_size = get_trader_max_size(trader)
        
        # Download data
        try:
            position_array   = get_positions(trader)     # Get the current position of the trader
            positions        = position_array["data"]["otherPositionRetList"]
            count = do_query(query="SELECT COUNT(*) as count FROM positions")["count"][0]
            if count > 0: request_id  = int(do_query(query="select MAX(id) as id from positions")["id"][0])
            else:         request_id = 0

        except:
            # print(f"Couldn't download position for {names[trader]}")
            continue #This is useful not to mix stuff between positions and no positions        
        
        # Insert in the database what we downloaded
        if not positions:
            # print(f"No positions for {names[trader]}, but still save.")
            empty    = [0,0,0,0,0,0,trader_max_size] # Need to get the timestamp right!
            insert_mysql(trader_info, request_id, empty, "CLOSE")

        if positions:
            for i in range(len(positions)):
                row = pd.DataFrame(positions).loc[i]
                vector = [row["symbol"], row["entryPrice"], row["markPrice"], row["pnl"], row["roe"], row["amount"], trader_max_size]
                status = "CLOSED" if row["amount"] == 0 else "OPEN"   # bug, this displays 0
                insert_mysql(trader_info, request_id, vector, status)
                
        
        # Get last updates
        two_ids = do_query(query=f"select distinct(request_id) from POSITIONS where encryptedUid = '{trader}' order by request_id desc limit 2")
        
        if len(two_ids)>=2: # if the trader is in the database, check the delta
            both = get_positions_db(two_ids)

            for i in range(len(both)):
                row = both.loc[i]
                message_logic(row, names[trader])
        
            time.sleep(2)
    
    #each loop
    if False: send_message(get_sentiment_message(), chat_id = -575092430)
        
    # print("####################################################################")
    # print("Time for one cycle of traders:", round( time.time() - start_trader,2))
    # print("####################################################################")

.....................message..............................
💰ClickHereNow 2022-01-06 08:27:48
*Modify existent trade*
Position change from 440384.0ADAUSDT to 440341.0ADAUSDT.
SELL 43.0ADAUSDT at 1.2155$
Trade went from 3.209% to 3.216% (SELL 0.007%)
.....................message..............................
💰ClickHereNow 2022-01-06 08:27:53
*Modify existent trade*
Position change from -387563.0XLMUSDT to -441168.0XLMUSDT.
SELL 53605.0XLMUSDT at 0.26889$
Trade went from -0.571% to -0.651% (SELL 0.08%)
.....................message..............................
💰ClickHereNow 2022-01-06 08:27:58
*Modify existent trade*
Position change from -2.654BTCUSDT to -5.524BTCUSDT.
SELL 2.87BTCUSDT at 43056.9$
Trade went from -0.69% to -1.435% (SELL 0.745%)
.....................message..............................
💰TradingHorse 2022-01-06 08:29:20
*Modify existent trade*
Position change from 119709.0MATICUSDT to 240956.0MATICUSDT.
BUY 121247.0MATICUSDT at 2.1336$
Trade went from 7.91% to 16.017% (BUY

In [ ]:
do_query(query="select MAX(id) as id from positions")["id"][0]

In [ ]:
do_query(query=f"select distinct(request_id) from POSITIONS where encryptedUid = '{trader}' order by request_id desc limit 2")

In [ ]:
do_query(query="select*from positions")